## Run Experiments in Parallel

In [1]:
import os
import sys
import typing
import json
import pandas as pd
if 'NOTEBOOKDIR' not in locals():
    NOTEBOOKDIR = os.getcwd()
from pathlib import Path
RUNDIR = str(Path(NOTEBOOKDIR).parent)
sys.path.append(str(RUNDIR))
import ray

In [2]:
# run "ray start --head --dashboard-host 0.0.0.0" from the repo root directory from within the venv lme.
# If you to attach another machine to the cluster, then run "ray start --address=<head-node-ip>:6379" there.
# To view dashboard, forward local port to remote dashboard either using vscode or via ssh: ssh -L 8265:<head-node-ip>:8265 <head-node-ip>
ray.init(address='auto')

2022-11-11 11:04:43,321	INFO worker.py:1333 -- Connecting to existing Ray cluster at address: 10.2.143.243:6379...
2022-11-11 11:04:43,328	INFO worker.py:1509 -- Connected to Ray cluster. View the dashboard at 10.2.143.243:8265 


Python version:,3.9.13
Ray version:,2.0.0
Dashboard:,http://10.2.143.243:8265


In [5]:
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

num_fewshots = [0, 5]
task_models = [('hellaswag_d', 'dist_sim')] #, ('hellaswag_d', 'dist_sim'), ('webqs_d', 'dist_sim')]
pretrained = ['EleutherAI/gpt-neo-1.3B']
encoding_schemes = ['segment_each_example'] # ['merge_all_segments', 'segment_each_example', 'concat_each_example', 'concat_all_examples']
word_agg_schemes = ['mean']
norms = ['layer']
sim_funcs = ['dot_product']
composition_func = 'soft_cluster'

@ray.remote(max_calls=1, num_gpus=1)
# @ray.remote(max_calls=1, num_cpus=4)
def run_eval(args):
    from main import main
    return main(*args)

futures = []
for num_fewshot in num_fewshots:
    for task, model in task_models:
        for submodel in pretrained:
            for encoding_scheme in encoding_schemes:
                for word_agg_scheme in word_agg_schemes:
                    for norm in norms:
                        for sim_func in sim_funcs:
                            _args = [
                                "--device", "0",
                                "--output_dir", "lmeval_results3/",
                                # "--limit", "5",
                                "--tasks", task,
                                "--model", model,
                                "--no_cache",
                                '--num_fewshot', f'{num_fewshot}',
                                '--task_args', f'encoding_scheme={encoding_scheme}'
                            ]
                            model_args = f'WORD_AGG_SCHEME={word_agg_scheme},EXAMPLE_AGG_SCHEME=None,SEGMENT_AGG_SCHEME=None,NORM={norm},SIMILARITY_FUNC={sim_func},COMPOSITION_FUNC={composition_func}'
                            if submodel is not None:
                                model_args = model_args + f',pretrained={submodel}'
                            _args.extend(['--model_args', model_args])
                            future = run_eval.remote(_args)
                            futures.append(future)

responses = ray.get(futures)
# for resp in responses:
#     fpath, results = resp
#     with open(fpath, "wt", encoding='utf-8') as f:
#         json.dump(results, f, indent=2)

responses

(run_eval pid=4594) 2022-11-11 11:27:27.651593: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


(run_eval pid=4594) Selected Tasks: ['hellaswag_d']
(run_eval pid=4594) Using device '0'


KeyboardInterrupt: 

(run_eval pid=4594) Using custom data configuration default
(run_eval pid=4594) Reusing dataset hellaswag (/home/ssingh/.cache/huggingface/datasets/hellaswag/default/0.1.0/c8c5bc30147e6345a39bfabe8856829801d4db7beb0271e44021811974fac112)
100%|██████████| 3/3 [00:00<00:00, 739.04it/s]


(run_eval pid=4594) Running distributed_encoding_similarity requests


  0%|          | 0/10042 [00:00<?, ?it/s]
(run_eval pid=4594) /home/ssingh/src/lm_eval_harness/lm_eval/models/dist_enc_model.py:234: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
(run_eval pid=4594)   scores = torch.nn.functional.softmax(scores)
  7%|▋         | 726/10042 [01:00<11:09, 13.92it/s]
